In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from estnltk.visualisation.span_visualiser.fancy_span_visualisation import DisplaySpans
from typing import Mapping, Any, Tuple, Sequence, Union
from estnltk import Text, Layer
from collections import defaultdict

In [3]:
def value_mapper_segment_type(segment: Tuple[str, Sequence], normal = '#ffffff00', conflicting = 'red', ambiguous = 'orange') -> str:
    if len(segment[1]) != 1:
        return conflicting
    elif len(segment[1][0].annotations) > 1:
        return ambiguous
    else: 
        return normal

In [4]:
class DisplayAmbiguousSpans(DisplaySpans):
    """
    Displays overlaps between spans and spans with multiple annotations
    
    Default background color scheme is following:
    * normal spans are transparent
    * overlapping spans are red
    * ambigious spans are orange
    The opacity level indicates the number of overlaps and annotations
    
    Color scheme is controlled by two dictionary like class attributes
    * span_coloring[int]
    * annotation_coloring[int]
    
    Assigning corresponding elements redefines the coloring for a particular
    number of annotations or spans, e.g. span_coloring[2] = 'blue'.
    
    Assigning corresponding attributes redefines the entire color scheme.
    The assigned object must support indexing with any int.
    """
    
    def __init__(self):
        super(DisplayAmbiguousSpans, self).__init__(styling="direct")
        
        # Define two shades of red for overlaps
        self.span_coloring = defaultdict(lambda:'#FF0000')
        self.span_coloring[2] = '#FF5050'

        # Define transparent + two shades of orange for ambigious annotations
        self.annotation_coloring = defaultdict(lambda:'#F59B00')
        self.annotation_coloring[1] = '#FFA50000'
        self.annotation_coloring[2] = '#FFA500'

        self.span_decorator.bg_mapping = self.__bg_mapper
        
    
    def __bg_mapper(self, segment: Tuple[str, Sequence]) -> str:
        if len(segment[1]) != 1:
            return self.span_coloring[segment[1]]
        
        return self.annotation_coloring[len(segment[1][0].annotations)]


In [5]:
class DisplayPostagsSpans(DisplaySpans):
    """
    Visualises different part of speech tags in a text
    
    Provides default background colourschme for EstMorf and GT tagsets.
    Color scheme is controlled by two dictionary like class attributes
    * pos_coloring[str]
    * span_coloring[int]
    
    The first coloring controls how spans with different pos tags are 
    colored. Default coloring can be changed by assigning appropriate
    entries, e.g. pos_coloring['V'] = 'black'.
    
    The second controls how span overaps are colored. The tokenisation 
    into the words can be ambiguous. By default overlaps are colored
    by two shades of read. This can be changed by assigning appropriate
    entries, e.g. span_coloring[2] = 'blue'.
    
    To redefine the entire color scheme the entire colouring attribute
    must be redefined. The assigned object must support indexing with 
    any string for pos_coloring and any int for span_coloring.
    
    As pos tagging may be ambiguous, colouring is done in two phases:
    1. list of pos tags is aggregated into a new string label
    2. pos tag coloring is used to determine the background color
    
    The default aggregator marks all ambigious labellings with '*'.
    It is possible to customise this by redefining ambiguity_resolver.
    """

    def __init__(self, layer:str='morph_analysis', tagset:str='EstMorf', ambiguity_resolver:callable=None):
        super(DisplayPostagsSpans, self).__init__(styling="direct")

        self.morph_layer = layer
        self.tagset = tagset
        self.pos_coloring = defaultdict(lambda:'#ffffff00')
        self.ambiguity_resolver = ambiguity_resolver or self.__default_resolver
        self.span_decorator.bg_mapping = self.__bg_mapper

        if self.tagset == 'EstMorf' or self.tagset == 'GT':
            self.pos_coloring['S'] = 'orange'
            self.pos_coloring['H'] = 'orange'
            self.pos_coloring['A'] = 'yellow'
            self.pos_coloring['U'] = 'yellow'
            self.pos_coloring['C'] = 'yellow'
            self.pos_coloring['N'] = 'yellow'
            self.pos_coloring['O'] = 'yellow'
            self.pos_coloring['V'] = 'lime'
            self.pos_coloring['*'] = 'gray'
            
        # Define two shades of red for overlaping tokenisation
        self.span_coloring = defaultdict(lambda:'#FF0000')
        self.span_coloring[2] = '#FF5050'
        
            
    def __call__(self, object:Union[Text, Layer]) -> str:
        if isinstance(object, Text):
            return super(DisplayPostagsSpans, self).__call__(object[self.morph_layer])
        elif isinstance(object, Layer):
            return super(DisplayPostagsSpans, self).__call__(object)
        else:
            raise ValueError('Invalid input')
            
            
    def __default_resolver(self, span) -> str:
        pos_tags = set(span['partofspeech'])
        if len(pos_tags) == 1:
            return next(iter(pos_tags));
        return '*'

    
    def __bg_mapper(self, segment: Tuple[str, Sequence]) -> str:
        if len(segment[1]) != 1:
            self.span_coloring[len(segment[1])]
        
        return self.pos_coloring[self.ambiguity_resolver(segment[1][0])]

In [6]:
class DisplayCompoundSpans(DisplaySpans):
    
    def __init__(self, layer:str='compound_tokens', ambiguity_resolver:callable=None):
        super(DisplayCompoundSpans, self).__init__(styling="direct")
        
        self.compound_layer = layer
        self.ambiguity_resolver = ambiguity_resolver or self.__default_resolver
        self.span_decorator.bg_mapping = self.__bg_mapper
        
        # Define two shades of red for overlaping tokenisation
        self.span_coloring = defaultdict(lambda:'#FF0000')
        self.span_coloring[2] = '#FF5050'

        # Define coloring for different types of compounds 
        self.type_coloring = defaultdict(lambda:'#ffffff00')
        
        # Proper names
        self.type_coloring['name_with_initial'] = '#6CA390'
        self.type_coloring['case_ending'] = '#6CA390'

        # Rare wordforms
        self.type_coloring['hyphenation'] = '#306754'
        
        # Numbers and units
        self.type_coloring['numeric_date'] = '#9DC209'
        self.type_coloring['numeric'] = '#9DC209'
        self.type_coloring['percentage'] = '#9DC209'
        self.type_coloring['unit'] = '#759A00'
        
        # Abrevations
        self.type_coloring['non_ending_abbreviation'] = '#BCE954'
        self.type_coloring['abbreviation'] = '#BCE954'
        
        # Web specific compounds
        self.type_coloring['xml_tag'] = '#5E5A80'
        self.type_coloring['email'] = '#908CB2'
        self.type_coloring['www_address'] = '#908CB2'
        
        # Emotiocons
        self.type_coloring['emoticon'] = '#FFDB58'
        
        # All the rest
        self.type_coloring['*'] = '#FFA62F'
        
        
    def __call__(self, object:Union[Text, Layer]) -> str:
        if isinstance(object, Text):
            return super(DisplayCompoundSpans, self).__call__(object[self.compound_layer])
        elif isinstance(object, Layer):
            return super(DisplayCompoundSpans, self).__call__(object)
        else:
            raise ValueError('Invalid input')
            
            
    def __default_resolver(self, span) -> str:
        types = set(span.type)
        types.discard('sign')
        if len(types) == 1:
            return next(iter(types));
        return '*'

    
    def __bg_mapper(self, segment: Tuple[str, Sequence]) -> str:
        if len(segment[1]) != 1:
            self.span_coloring[len(segment[1])]
            
        return self.type_coloring[self.ambiguity_resolver(segment[1][0])]    

In [7]:
from estnltk import Text
from estnltk.taggers import TokensTagger
from estnltk.taggers import CompoundTokenTagger

In [32]:
raw_text  = 'Mis aias sa-das 3me sorti s-saia?\n'
raw_text += '02.02.2010 22:55 Mati : saad sa mulle 100,50 asemel 10 000 laenata?\n'
raw_text += 'Mati : +100% kindel, et toon tagasi!!\n'
raw_text += 'Tänase seisuga tuleb ikka suur lohe vaiksema tuule (6-12 m/s) jaoks ja teine väiksem tormikaks (12-20 m/s) võtta…\n'
raw_text += '<u>Kirjavahemärgid, hingamiskohad</u>.\n'
raw_text += 'Saada need e-postiaadressile big@boss.com või tule sisesta lehelt www.iamboss.com\n'
raw_text += 'Maja on fantastiline :)) ja mõte on hea :-)\n'
raw_text += '(arhitektid M. Port, M. Meelak, O. Zhemtshugov, R.-L. Kivi)\n'
raw_text += 'Nt. hädas oli juba Vana-Hiina suurim ajaloolane Sima Qian (II—I saj. e. m. a.).\n'
raw_text += "10 000-st LinkedIn 'i kontaktist mitte üks ei hoolinud meie SKT -st, aga meie workshop ' e väisasid küll\n."
text = TokensTagger().tag(Text(raw_text))
CompoundTokenTagger().tag(text)
display_compounds = DisplayCompoundSpans()
display_compounds(text.compound_tokens)

Mis aias sa-das 3me sorti s-saia ? 02.02.2010 22:55 Mati : saad sa mulle 100,50 asemel 10 000 laenata? Mati : +100% kindel, et toon tagasi!! Tänase seisuga tuleb ikka suur lohe vaiksema tuule (6-12 m/s ) jaoks ja teine väiksem tormikaks (12-20 m/s ) võtta… <u> Kirjavahemärgid, hingamiskohad </u> . Saada need e-postiaadressile big@boss.com või tule sisesta lehelt www.iamboss.com Maja on fantastiline :)) ja mõte on hea :-) (arhitektid M. Port , M. Meelak , O. Zhemtshugov , R.-L. Kivi ) Nt. hädas oli juba Vana-Hiina suurim ajaloolane Sima Qian (II—I saj. e. m. a. ). 10 000-st LinkedIn 'i kontaktist mitte üks ei hoolinud meie SKT -st , aga meie workshop ' e väisasid küll .

* Short_Introduction tutorial: saaks compound_token'ite puhul tuua esile, millised token'id nende sees on, nt vahelduvate värvidega -- heledad ja tumedad toonid -- sõna sees;


* CompoundTokenTagger'i tutorial-is saaks lihtsalt värvida compound_token'eid, et eristada neid ülejäänud sõnadest. Compound_token'ite eri tüüpide (a la 'hyphenation','email','emoticon' jne) eraldi värvimine praeguste näidete puhul aga minumeelest midagi juurde ei annaks; 

* SentenceTokenizer'i tutorial-is saaks põhimõtteliselt värvidega näidata, milline on järelparanduste mõju, st millised olid vanad lauselõpumärgid (sõnad) ja millised on uued. Nt eristada värvidega ainult vanu lauselõpumärke, ainult uusi lauselõpumärke ning lauselõpumärke, mis langevad vanas ja uues kokku. Selleks peab aga 2 lausestuse kihti tekitama (järelparandusega vs ilma) ning kui teha seda tutoriali sees, siis on oht, et kood läheb keerulisemaks, kui on mugav jälgida;

* ParagraphTokenizer: saab näidata paragrahvide ja lausete suhet: millised laused paiknevad paragrahvi sees. Värvida tuleks paragrahve/lauseid enclosing_text'i järgi;

* Morf analüüsi colorgames.html näited on minumeelest üsna head. Lisada saaks veel leksikonist väljajäävate / tundmatute sõnade värvimise -- siis, kui oletamine ja ühestamine välja lülitada;

* ClauseSegmenter'i puhul laused ühte värvi, osalaused teist värvi, et oleks näha, kuidas laused jagunevad osalauseteks. Mina värviks jällegi enclosing_text'e;
* SyntaxIgnoreTagger'i puhul saaks lihtsalt värvidega esile tuua ignoreeritavad tekstilõigud;

* Kui toimub värvimine, kas saaks värvitud teksti alla (või kõrvale) automaatselt tekitada ka tabeli, kus on nn värvide legend -- et mida mis värviga esile tuuakse? See võiks olla selline optional asi, aga ehk kulub väljundi selgitamisel ära;

In [10]:
t=Text("""
Silver Ükssilma lugu

Silver, kus on sinu kullamäed? 
Merepõhja vara maha jäi koos laevaga. 

Silver, kus on sinu julged teod? 
Merepõhja noorus maha jäi koos laevaga. 

Silver, kus on sinu vasak silm? 
Merepõhja silm maha jäi koos laevaga. 

Silver, kus on sinu röövlisalk? 
Merepõhja poisid maha jäid koos laevaga. 

Refr. Üksi ma veel kõrtsu laua taga, 
piigad põlvedel oma viimseid päevi magan. 

Silver, kus on sinu poisipõnn? 
Merel seilab minu poisipõnn musta lipu all. 

Silver, kus on sinu sünnipaik? 
Laevakajutis mu sünnipaik musta lipu all. 

Silver, kus on sinu õige koht? 
Laeva tekil minu õige koht musta lipu all. 

Silver, kus on sinu röövlisalk? 
Merepõhja poisid maha jäid koos laevaga. 

Refr. Üksi ma veel … 

Merel sõidab minu poisipõnn, 
ahhoi!
""")
t.analyse('morphology')

Text(text='\nSilver Ükssilma lugu\n\nSilver, kus on sinu kullamäed? \nMerepõhja vara maha jäi koos laevaga. \n\nSilver, kus on sinu julged teod? \nMerepõhja noorus maha jäi koos laevaga. \n\nSilver, kus on sinu vasak silm? \nMerepõhja silm maha jäi koos laevaga. \n\nSilver, kus on sinu röövlisalk? \nMerepõhja poisid maha jäid koos laevaga. \n\nRefr. Üksi ma veel kõrtsu laua taga, \npiigad põlvedel oma viimseid päevi magan. \n\nSilver, kus on sinu poisipõnn? \nMerel seilab minu poisipõnn musta lipu all. \n\nSilver, kus on sinu sünnipaik? \nLaevakajutis mu sünnipaik musta lipu all. \n\nSilver, kus on sinu õige koht? \nLaeva tekil minu õige koht musta lipu all. \n\nSilver, kus on sinu röövlisalk? \nMerepõhja poisid maha jäid koos laevaga. \n\nRefr. Üksi ma veel … \n\nMerel sõidab minu poisipõnn, \nahhoi!\n')

In [11]:
display_ambigous = DisplayAmbiguousSpans()
display_ambigous(t.morph_analysis)


Silver Ükssilma lugu

Silver, kus on sinu kullamäed? 
Merepõhja vara maha jäi koos laevaga. 

Silver, kus on sinu julged teod? 
Merepõhja noorus maha jäi koos laevaga. 

Silver, kus on sinu vasak silm? 
Merepõhja silm maha jäi koos laevaga. 

Silver, kus on sinu röövlisalk? 
Merepõhja poisid maha jäid koos laevaga. 

Refr. Üksi ma veel kõrtsu laua taga, 
piigad põlvedel oma viimseid päevi magan. 

Silver, kus on sinu poisipõnn? 
Merel seilab minu poisipõnn musta lipu all. 

Silver, kus on sinu sünnipaik? 
Laevakajutis mu sünnipaik musta lipu all. 

Silver, kus on sinu õige koht? 
Laeva tekil minu õige koht musta lipu all. 

Silver, kus on sinu röövlisalk? 
Merepõhja poisid maha jäid koos laevaga. 

Refr. Üksi ma veel … 

Merel sõidab minu poisipõnn, 
ahhoi!



Silver Ükssilma lugu Silver , kus on sinu kullamäed ? Merepõhja vara maha jäi koos laevaga . Silver , kus on sinu julged teod ? Merepõhja noorus maha jäi koos laevaga . Silver , kus on sinu vasak silm ? Merepõhja silm maha jäi koos laevaga . Silver , kus on sinu röövlisalk ? Merepõhja poisid maha jäid koos laevaga . Refr . Üksi ma veel kõrtsu laua taga , piigad põlvedel oma viimseid päevi magan . Silver , kus on sinu poisipõnn ? Merel seilab minu poisipõnn musta lipu all . Silver , kus on sinu sünnipaik ? Laevakajutis mu sünnipaik musta lipu all . Silver , kus on sinu õige koht ? Laeva tekil minu õige koht musta lipu all . Silver , kus on sinu röövlisalk ? Merepõhja poisid maha jäid koos laevaga . Refr . Üksi ma veel … Merel sõidab minu poisipõnn , ahhoi !

In [12]:
display_postags = DisplayPostagsSpans()
display_postags(t.morph_analysis)
display_postags(t)


Silver Ükssilma lugu

Silver, kus on sinu kullamäed? 
Merepõhja vara maha jäi koos laevaga. 

Silver, kus on sinu julged teod? 
Merepõhja noorus maha jäi koos laevaga. 

Silver, kus on sinu vasak silm? 
Merepõhja silm maha jäi koos laevaga. 

Silver, kus on sinu röövlisalk? 
Merepõhja poisid maha jäid koos laevaga. 

Refr. Üksi ma veel kõrtsu laua taga, 
piigad põlvedel oma viimseid päevi magan. 

Silver, kus on sinu poisipõnn? 
Merel seilab minu poisipõnn musta lipu all. 

Silver, kus on sinu sünnipaik? 
Laevakajutis mu sünnipaik musta lipu all. 

Silver, kus on sinu õige koht? 
Laeva tekil minu õige koht musta lipu all. 

Silver, kus on sinu röövlisalk? 
Merepõhja poisid maha jäid koos laevaga. 

Refr. Üksi ma veel … 

Merel sõidab minu poisipõnn, 
ahhoi!



Silver Ükssilma lugu Silver , kus on sinu kullamäed ? Merepõhja vara maha jäi koos laevaga . Silver , kus on sinu julged teod ? Merepõhja noorus maha jäi koos laevaga . Silver , kus on sinu vasak silm ? Merepõhja silm maha jäi koos laevaga . Silver , kus on sinu röövlisalk ? Merepõhja poisid maha jäid koos laevaga . Refr . Üksi ma veel kõrtsu laua taga , piigad põlvedel oma viimseid päevi magan . Silver , kus on sinu poisipõnn ? Merel seilab minu poisipõnn musta lipu all . Silver , kus on sinu sünnipaik ? Laevakajutis mu sünnipaik musta lipu all . Silver , kus on sinu õige koht ? Laeva tekil minu õige koht musta lipu all . Silver , kus on sinu röövlisalk ? Merepõhja poisid maha jäid koos laevaga . Refr . Üksi ma veel … Merel sõidab minu poisipõnn , ahhoi !


Silver Ükssilma lugu

Silver, kus on sinu kullamäed? 
Merepõhja vara maha jäi koos laevaga. 

Silver, kus on sinu julged teod? 
Merepõhja noorus maha jäi koos laevaga. 

Silver, kus on sinu vasak silm? 
Merepõhja silm maha jäi koos laevaga. 

Silver, kus on sinu röövlisalk? 
Merepõhja poisid maha jäid koos laevaga. 

Refr. Üksi ma veel kõrtsu laua taga, 
piigad põlvedel oma viimseid päevi magan. 

Silver, kus on sinu poisipõnn? 
Merel seilab minu poisipõnn musta lipu all. 

Silver, kus on sinu sünnipaik? 
Laevakajutis mu sünnipaik musta lipu all. 

Silver, kus on sinu õige koht? 
Laeva tekil minu õige koht musta lipu all. 

Silver, kus on sinu röövlisalk? 
Merepõhja poisid maha jäid koos laevaga. 

Refr. Üksi ma veel … 

Merel sõidab minu poisipõnn, 
ahhoi!



Silver Ükssilma lugu Silver , kus on sinu kullamäed ? Merepõhja vara maha jäi koos laevaga . Silver , kus on sinu julged teod ? Merepõhja noorus maha jäi koos laevaga . Silver , kus on sinu vasak silm ? Merepõhja silm maha jäi koos laevaga . Silver , kus on sinu röövlisalk ? Merepõhja poisid maha jäid koos laevaga . Refr . Üksi ma veel kõrtsu laua taga , piigad põlvedel oma viimseid päevi magan . Silver , kus on sinu poisipõnn ? Merel seilab minu poisipõnn musta lipu all . Silver , kus on sinu sünnipaik ? Laevakajutis mu sünnipaik musta lipu all . Silver , kus on sinu õige koht ? Laeva tekil minu õige koht musta lipu all . Silver , kus on sinu röövlisalk ? Merepõhja poisid maha jäid koos laevaga . Refr . Üksi ma veel … Merel sõidab minu poisipõnn , ahhoi !

In [13]:
%config IPCompleter.greedy=True

In [14]:
def display_ambigious(layer, bg_color = 'orange'):
    """
    Colors all ambigious spans  
    """
    
    disp2 = DisplaySpans(styling="direct")
    disp2.span_decorator.bg_mapping = lambda segment: color_first_cases(segment, color_dict, ambiguous)
    disp2(t.morph_analysis)    

In [15]:
def color_postags(segment: Tuple[str, Sequence],  color_dict, ambiguous ) -> str:
    """
    Colors based on postags
    """

    if len(segment[1]) != 1:
        
        return 'red'
    
    # It might be better to convert it to string     
    pos_tags = getattr(segment[1][0], 'partofspeech')

    # Ambigous POS tagging
    
    postags_set = sorted(list(set(pos_tags)))
    
    if len(postags_set) > 1:
        return ambiguous

    return color_dict.get(pos_tags[0], 'white')

In [16]:
def display_postags(text, color_dict= {'S': 'orange', 'H': 'orange', 'A': 'yellow', 'U': 'yellow', 'C': 'yellow', 'N': 'yellow', 
                       'O': 'yellow', 'V': 'lime'}, ambiguous= 'white'):
    
    """
    Displays some postags in different colors. Colors and postags can be defined by user
    """
    
    t = Text(text).tag_layer()
    disp2 = DisplaySpans(styling="direct")
    disp2.span_decorator.bg_mapping = lambda segment: color_postags(segment, color_dict, ambiguous)
    disp2(t.morph_analysis)


In [17]:
display_postags('Samojeedid on valged karvapallid.')

Samojeedid on valged karvapallid.


Samojeedid on valged karvapallid .

In [18]:
display_postags('Partitsiibid on jäänud  mitmeseks.')

Partitsiibid on jäänud  mitmeseks.


Partitsiibid on jäänud mitmeseks .

In [19]:
color_dict = {'S': 'lightgreen', 'H': 'lightgreen', 'A': 'pink'}

In [20]:
display_postags('Samojeedid on valged karvapallid.', color_dict = color_dict)

Samojeedid on valged karvapallid.


Samojeedid on valged karvapallid .

In [21]:
display_postags('Partitsiibid on jäänud  mitmeseks.', ambiguous = 'red')

Partitsiibid on jäänud  mitmeseks.


Partitsiibid on jäänud mitmeseks .

In [22]:
def color_first_cases(segment: Tuple[str, Sequence], color_dict, ambiguous ) -> str:
    """
    Colors the first 3 and a half cases
    """
    #print(segment[1])
    # Tokenization conflict
    if len(segment[1]) != 1:
        
        return 'red'
    
    # It might be better to convert it to string     
    #pos_tags = getattr(segment[1][0], 'partofspeech')
    forms = getattr(segment[1][0], 'form')
    # Ambigous POS tagging
    
    forms_set = sorted(list(set(forms)))
    
    if len(forms) > 1:
        return 'red'

    color_dict = {'sg n': 'orange', 'pl n': 'orange', 'sg g': 'yellow', 'pl g': 'yellow', 'sg p': 'lightgreen',
                 'pl p': 'lightgreen', 'adt': 'pink'}
    
    return color_dict.get(forms_set[0], 'white')

In [23]:
def display_first_cases(text, color_dict = {'sg n': 'orange', 'pl n': 'orange', 'sg g': 'yellow', 'pl g': 'yellow', 'sg p': 'lightgreen',
                 'pl p': 'lightgreen', 'adt': 'pink'}, ambiguous = 'red'):
    """
    Displays the first 3 and a half cases in different colors, can be defined by user
    """
    t = Text(text).tag_layer()
    disp2 = DisplaySpans(styling="direct")
    disp2.span_decorator.bg_mapping = lambda segment: color_first_cases(segment, color_dict, ambiguous)
    disp2(t.morph_analysis)


In [24]:
display_first_cases('aias sadas saia ja leiva peale kukkus õun ja vette hüppasid surnud kalad')

aias sadas saia ja leiva peale kukkus õun ja vette hüppasid surnud kalad


aias sadas saia ja leiva peale kukkus õun ja vette hüppasid surnud kalad

In [25]:
def color_compounds(segment: Tuple[str, Sequence]) -> str:
    """
    Colors compound words in millennial pink
    """

    # Tokenization conflict
    if len(segment[1]) != 1:
        
        return 'green'
    
    root_tokens = getattr(segment[1][0], 'root_tokens')
 
    for token in root_tokens:
        if len(token) > 1:
            return '#ffb6c1'

    color_dict = {}

    return color_dict.get(root_tokens[0], 'white')

In [26]:
def display_compounds(text):
    """
    Displays compound words
    """
    t = Text(text).tag_layer()
    disp3 = DisplaySpans(styling="direct")
    disp3.span_decorator.bg_mapping = lambda segment: color_compounds(segment)
    disp3(t.morph_analysis)

In [27]:
display_compounds('rukkililled ja jääkarud on tavalised liitsõnad, aga roosid ja rebased mitte')

rukkililled ja jääkarud on tavalised liitsõnad, aga roosid ja rebased mitte


rukkililled ja jääkarud on tavalised liitsõnad , aga roosid ja rebased mitte